## Data Preprocessing(SQL Based)

In [1]:
import pandas as pd
import numpy as np
from pandasql import sqldf

In [2]:
df = pd.read_csv('csv_flight/combined_df.csv')
df.head()

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Reporting_Airline,Tail_Number,Flight_Number_Reporting_Airline,OriginAirportSeqID,...,Dest,DestCityName,DestState,CRSDepTime,DepDelay,CRSArrTime,Cancelled,Diverted,CRSElapsedTime,Distance
0,2014,1,1,26,7,2014-01-26,DL,N366NW,2465,1039705,...,IAH,"Houston, TX",TX,2145,-2.0,2305,0.0,0.0,140.0,689.0
1,2014,1,1,26,7,2014-01-26,DL,N951DL,2474,1129803,...,ATL,"Atlanta, GA",GA,945,-8.0,1244,0.0,0.0,119.0,731.0
2,2014,1,1,26,7,2014-01-26,DL,N929DN,2491,1039705,...,DEN,"Denver, CO",CO,1855,-2.0,2026,0.0,0.0,211.0,1199.0
3,2014,1,1,26,7,2014-01-26,DL,N666DN,2500,1039705,...,PHX,"Phoenix, AZ",AZ,1634,-1.0,1851,0.0,0.0,257.0,1587.0
4,2014,1,1,26,7,2014-01-26,DL,N682DA,2504,1410702,...,ATL,"Atlanta, GA",GA,700,-7.0,1227,0.0,0.0,207.0,1587.0


In [3]:
df.shape

(1658130, 24)

### Handle Missing Values

In [4]:
# check percentages of NaN in each column
nan_percentage = df.isnull().mean()
nan_percentage

Year                               0.000000
Quarter                            0.000000
Month                              0.000000
DayofMonth                         0.000000
DayOfWeek                          0.000000
FlightDate                         0.000000
Reporting_Airline                  0.000000
Tail_Number                        0.003188
Flight_Number_Reporting_Airline    0.000000
OriginAirportSeqID                 0.000000
Origin                             0.000000
OriginCityName                     0.000000
OriginState                        0.000000
DestAirportSeqID                   0.000000
Dest                               0.000000
DestCityName                       0.000000
DestState                          0.000000
CRSDepTime                         0.000000
DepDelay                           0.010979
CRSArrTime                         0.000000
Cancelled                          0.000000
Diverted                           0.000000
CRSElapsedTime                  

Only Tail_Number  and DepDelay have missing values with a small proportion. We will just remove rows with missing values.

In [5]:
# sql query to select all non-Null rows
sql_query = """
SELECT *
FROM df
WHERE Tail_Number IS NOT NULL AND DepDelay IS NOT NULL
"""

df = sqldf(sql_query, globals())

In [6]:
print(df.isnull().any().any())
print(df.shape)

False
(1639925, 24)


All missing values are removed.

### Handle Dublicated Values

In [7]:
# check the proportion of dublicated rows
proportion_duplicated = df.duplicated(keep='first').mean()
print(f"Proportion of duplicated rows: {proportion_duplicated:.2%}")

Proportion of duplicated rows: 0.00%


There is no dublicated rows in the dataframe.

### Create Federal Holidays Variable

In [8]:
# create a new variable DaysSince: number of days since 2014-01-01 for each FlightDate
ref_date = pd.to_datetime('2014-01-01')
df['DaysSince'] = (pd.to_datetime(df['FlightDate']) - ref_date).dt.days
print("Length of Unique DaySince:", len(df['DaysSince'].unique())) # there are 1826 days between 2014-01-01 to 2018-12-31

Length of Unique DaySince: 1826


In [9]:
# sort dataframe by Origin then DaysSince then CRSDepTime
sql_query = """
SELECT *
FROM df
ORDER BY LOWER(Origin), DaysSince, CRSDepTime
"""

df = sqldf(sql_query, globals())

In [10]:
# federal holidays since 2014-01-01 to 2018-12-31
holidays = [
    '2014-01-01', '2014-01-20', '2014-02-17', '2014-05-26', '2014-07-04', '2014-09-01', 
    '2014-10-13', '2014-11-11', '2014-11-27', '2014-12-25', '2015-01-01', '2015-01-19', 
    '2015-02-16', '2015-05-25', '2015-07-03', '2015-09-07', '2015-10-12', '2015-11-11', 
    '2015-11-26', '2015-12-25', '2016-01-01', '2016-01-18', '2016-02-15', '2016-05-30', 
    '2016-07-04', '2016-09-05', '2016-10-10', '2016-11-11', '2016-11-24', '2016-12-25', 
    '2017-01-01', '2017-01-16', '2017-02-20', '2017-05-29', '2017-07-04', '2017-09-04', 
    '2017-10-09', '2017-11-10', '2017-11-23', '2017-12-25', '2018-01-01', '2018-01-15', 
    '2018-02-19', '2018-05-28', '2018-07-04', '2018-09-03', '2018-10-08', '2018-11-11', 
    '2018-11-22', '2018-12-25'
]

In [11]:
# convert holiday date to number of days since 2014-01-01
holiday_since = (pd.to_datetime(holidays) - ref_date).days
holiday_since_df = pd.DataFrame({'HolidaySince': holiday_since})

In [12]:
# sql query to create two new columns: is_holiday and is_day_before_holiday
globals_dict = globals()
globals_dict['df'] = df
globals_dict['holiday_since_df'] = holiday_since_df

sql_query = """
SELECT df.*, 
       CASE WHEN hs.HolidaySince IS NOT NULL THEN 1 ELSE 0 END as is_holiday,
       CASE WHEN hs_before.HolidaySince IS NOT NULL THEN 1 ELSE 0 END as is_day_before_holiday
FROM df
LEFT JOIN holiday_since_df hs ON df.DaysSince = hs.HolidaySince
LEFT JOIN holiday_since_df hs_before ON df.DaysSince = hs_before.HolidaySince - 1
"""

df = sqldf(sql_query, globals_dict)

In [13]:
# sql query to create a new columns: is_holiday_week if the flight day is in a 
# holliday week(3 days before and 3 days after).
globals_dict = globals()
globals_dict['df'] = df
globals_dict['holiday_since_df'] = holiday_since_df

sql_query = """
SELECT df.*,
       CASE WHEN EXISTS (
            SELECT 1
            FROM holiday_since_df hs
            WHERE df.DaysSince BETWEEN hs.HolidaySince - 3 AND hs.HolidaySince + 3
        ) THEN 1 ELSE 0 END as is_holiday_week
FROM df
"""

df = sqldf(sql_query, globals_dict)
df.head()

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Reporting_Airline,Tail_Number,Flight_Number_Reporting_Airline,OriginAirportSeqID,...,DepDelay,CRSArrTime,Cancelled,Diverted,CRSElapsedTime,Distance,DaysSince,is_holiday,is_day_before_holiday,is_holiday_week
0,2014,1,1,1,3,2014-01-01,OO,N724SK,4974,1039705,...,-3.0,721,0.0,0.0,141.0,689.0,0,1,0,1
1,2014,1,1,1,3,2014-01-01,AA,N002AA,1315,1039705,...,-2.0,830,0.0,0.0,150.0,731.0,0,1,0,1
2,2014,1,1,1,3,2014-01-01,UA,N76288,1134,1039705,...,2.0,815,0.0,0.0,128.0,606.0,0,1,0,1
3,2014,1,1,1,3,2014-01-01,DL,N709DN,110,1039705,...,21.0,1004,0.0,0.0,294.0,1947.0,0,1,0,1
4,2014,1,1,1,3,2014-01-01,DL,N654DL,1347,1039705,...,-2.0,1038,0.0,0.0,323.0,2139.0,0,1,0,1


### Create Origin Flight Density Variable

In [14]:
# create a new column MinSince: number of minutes since 2014-01-01 0:00 for each CRSDepTime
df['MinSince'] = df['DaysSince'] * 1440 + (df['CRSDepTime'] // 100) * 60 + (df['CRSDepTime'] % 100)

In [15]:
def calculate_density(sliding_window, time_before, time_after):
    """
    Calculate the density of flights within the time window for each flight
    """
    n = len(sliding_window)
    counts = np.zeros(n)
    start_idx = 0
    end_idx = 0

    for i in range(n):
        # move the start index forward if the flight at start_idx is outside the current window's start
        while sliding_window[start_idx] < sliding_window[i] - time_before:
            start_idx += 1

        # Move the end index forward if the flight at end_idx is inside the current window's end
        while end_idx < n and sliding_window[end_idx] <= sliding_window[i] + time_after:
            end_idx += 1

        # the count for the current flight's window is the difference between end_idx and start_idx
        counts[i] = end_idx - start_idx

    return counts

# list to store flight density results
density_results = []

# group by Origin and process each group
for origin, group in df.groupby('Origin'):
    sliding_window = group['MinSince'].values
    # 120 minutes before and 60 minutes after
    density = calculate_density(sliding_window, 120, 60)
    density_results.extend(density)


df['OriginFlightDensity'] = density_results

df.head()

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Reporting_Airline,Tail_Number,Flight_Number_Reporting_Airline,OriginAirportSeqID,...,Cancelled,Diverted,CRSElapsedTime,Distance,DaysSince,is_holiday,is_day_before_holiday,is_holiday_week,MinSince,OriginFlightDensity
0,2014,1,1,1,3,2014-01-01,OO,N724SK,4974,1039705,...,0.0,0.0,141.0,689.0,0,1,0,1,360,2.0
1,2014,1,1,1,3,2014-01-01,AA,N002AA,1315,1039705,...,0.0,0.0,150.0,731.0,0,1,0,1,420,3.0
2,2014,1,1,1,3,2014-01-01,UA,N76288,1134,1039705,...,0.0,0.0,128.0,606.0,0,1,0,1,427,3.0
3,2014,1,1,1,3,2014-01-01,DL,N709DN,110,1039705,...,0.0,0.0,294.0,1947.0,0,1,0,1,490,6.0
4,2014,1,1,1,3,2014-01-01,DL,N654DL,1347,1039705,...,0.0,0.0,323.0,2139.0,0,1,0,1,495,6.0


### Create Weather Variables

Weather data were collected from Local Climatological Data Station Details Data on NOAA(National Centers for Environmental Information) website. For each airport, data was downloaded on:

ATL(ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPORT, GA US): https://www.ncdc.noaa.gov/cdo-web/datasets/LCD/stations/WBAN:13874/detail

ORD(CHICAGO OHARE INTERNATIONAL AIRPORT, IL US): https://www.ncdc.noaa.gov/cdo-web/datasets/LCD/stations/WBAN:94846/detail

DFW(DALLAS FAA AIRPORT, TX US): https://www.ncdc.noaa.gov/cdo-web/datasets/LCD/stations/WBAN:13960/detail

DEN(DENVER INTERNATIONAL AIRPORT, CO US): https://www.ncdc.noaa.gov/cdo-web/datasets/LCD/stations/WBAN:13960/detail

CLT(CHARLOTTE DOUGLAS AIRPORT, NC US): https://www.ncdc.noaa.gov/cdo-web/datasets/LCD/stations/WBAN:13881/detail

LAX(LOS ANGELES INTERNATIONAL AIRPORT, CA US): https://www.ncdc.noaa.gov/cdo-web/datasets/LCD/stations/WBAN:23174/detail

IAH(HOUSTON INTERCONTINENTAL AIRPORT, TX US): https://www.ncdc.noaa.gov/cdo-web/datasets/LCD/stations/WBAN:12960/detail

PHX(PHOENIX AIRPORT, AZ US): https://www.ncdc.noaa.gov/cdo-web/datasets/LCD/stations/WBAN:23183/detail

SFO(SAN FRANCISCO INTERNATIONAL AIRPORT, CA US): https://www.ncdc.noaa.gov/cdo-web/datasets/LCD/stations/WBAN:23234/detail

Visibility: The horizontal distance an object can be seen and identified given in whole miles. Note visibilities less than 3 miles are usually given in smaller increments (e.g. 2.5).

Wind Speed: Speed of the wind at the time of observation given in miles per hour (mph).

In [16]:
# historical weather of Hartsfield-Jackson Atlanta International Airport
ATL_weather = pd.read_csv('csv_weather/atl.csv', low_memory=False)
# use HourlyVisibility and HourlyWindSpeed as weather indicators
ATL_weather = ATL_weather[['DATE', 'REPORT_TYPE', 'HourlyVisibility', 'HourlyWindSpeed']]
ATL_weather.head()

,DATE,REPORT_TYPE,HourlyVisibility,HourlyWindSpeed
0,2014-01-01T00:52:00,FM-15,10.00,0.0
1,2014-01-01T01:00:00,FM-12,9.94,0.0
2,2014-01-01T01:52:00,FM-15,10.00,3.0
3,2014-01-01T02:52:00,FM-15,10.00,0.0
4,2014-01-01T03:52:00,FM-15,10.00,0.0


In [17]:
# only use METAR Format(FM-15) data
ATL_weather = ATL_weather[ATL_weather['REPORT_TYPE'] == 'FM-15']

In [18]:
# create a new column MinSince: number of minutes since 2014-01-01 0:00 for each weather record
ref_datetime = pd.to_datetime('2014-01-01 00:00')
ATL_weather['MinSince'] = (pd.to_datetime(ATL_weather['DATE']) - ref_datetime).dt.total_seconds() / 60
ATL_weather.head()

,DATE,REPORT_TYPE,HourlyVisibility,HourlyWindSpeed,MinSince
0,2014-01-01T00:52:00,FM-15,10.00,0.0,52.0
2,2014-01-01T01:52:00,FM-15,10.00,3.0,112.0
3,2014-01-01T02:52:00,FM-15,10.00,0.0,172.0
4,2014-01-01T03:52:00,FM-15,10.00,0.0,232.0
5,2014-01-01T04:52:00,FM-15,10.00,3.0,292.0


In [19]:
# map weather data to each flight row based on the closest MinSince value
def map_closest_weather(df, weather_df, airport_code):
    if 'Visibility' not in df.columns:
        df['Visibility'] = None
    if 'WindSpeed' not in df.columns:
        df['WindSpeed'] = None
    
    # filter flight dataframe for the specific airport
    df_airport = df[df['Origin'] == airport_code].copy()
    
    # indices for iterating through the weather dataframe
    weather_idx = 0
    weather_max_idx = len(weather_df) - 1
    
    # iterate through each flight in the filtered flight dataframe
    for idx, flight in df_airport.iterrows():
        # advance the weather index to find the closest prior weather report
        while weather_idx < weather_max_idx and weather_df.iloc[weather_idx + 1]['MinSince'] <= flight['MinSince']:
            weather_idx += 1
            
        # assign the closest weather data to the flight dataframe
        closest_weather = weather_df.iloc[weather_idx]
        df.at[idx, 'Visibility'] = closest_weather['HourlyVisibility']
        df.at[idx, 'WindSpeed'] = closest_weather['HourlyWindSpeed']
    
    return df

In [20]:
df = map_closest_weather(df, ATL_weather, 'ATL')
df[['Origin', 'MinSince', 'Visibility', 'WindSpeed']].head()

,Origin,MinSince,Visibility,WindSpeed
0,ATL,360,10.00,5.0
1,ATL,420,10.00,5.0
2,ATL,427,10.00,5.0
3,ATL,490,10.00,3.0
4,ATL,495,10.00,3.0


In [21]:
# apply weather data mapping to all airports
airport_code_list = ['CLT', 'DEN', 'DFW', 'IAH', 'LAX', 'ORD', 'PHX', 'SFO']
ref_datetime = pd.to_datetime('2014-01-01 00:00')

In [22]:
for airport_code in airport_code_list:
    weather_df = pd.read_csv(f'csv_weather/{airport_code}.csv', low_memory=False)
    weather_df = weather_df[['DATE', 'REPORT_TYPE', 'HourlyVisibility', 'HourlyWindSpeed']]
    weather_df = weather_df[weather_df['REPORT_TYPE'] == 'FM-15']
    weather_df['MinSince'] = (pd.to_datetime(weather_df['DATE']) - ref_datetime).dt.total_seconds() / 60
    df = map_closest_weather(df, weather_df, airport_code)

In [23]:
df[['Origin', 'MinSince', 'Visibility', 'WindSpeed']].tail()

,Origin,MinSince,Visibility,WindSpeed
1639920,SFO,2629200,10.00,28.0
1639921,SFO,2629365,10.00,32.0
1639922,SFO,2629365,10.00,32.0
1639923,SFO,2629385,10.00,28.0
1639924,SFO,2629425,10.00,28.0


In [24]:
# check unique values of Visibility
unique_vis = df['Visibility'].unique()
print("Unique Visibility Values:", unique_vis)

Unique Visibility Values: ['10.00' '1.00' '0.50V' '0.50' '2.50' '8.00' '0.75' '3.00' '7.00' '4.00'
 '2.00' '9.00' '0.25' '1.50' '6.00' '1.25' '5.00' '1.75' '0.12' '1.50V'
 '0.00' '2.00V' '0.75V' '3.00V' '2.50V' '4.00V' '5.00V' '0.25V' '1.00V'
 nan '1.25V' '15.00' '7.00V' '1.75V' '0.00V']


In [25]:
# proportion of NA value of Visibility
vis_na_proportion = df['Visibility'].isna().mean()
print("Proportion of Visibility NA values:", vis_na_proportion)

Proportion of Visibility NA values: 6.768602222662622e-05


In [26]:
# remove rows with Visibility is NA
df = df.dropna(subset=['Visibility'])

# convert Visibility to float type
df['Visibility'] = df['Visibility'].str.replace('V', '', regex=False)
df['Visibility'] = df['Visibility'].astype(float)

In [27]:
# check unique values of WindSpeed
unique_ws = df['WindSpeed'].unique()
print("Unique WindSpeed Values:", unique_ws)

Unique WindSpeed Values: [5.0 3.0 6.0 0.0 7.0 8.0 13.0 21.0 28.0 22.0 24.0 29.0 15.0 14.0 11.0 9.0
 10.0 17.0 25.0 20.0 18.0 23.0 16.0 26.0 31.0 33.0 30.0 nan nan nan nan
 nan nan nan nan nan 38.0 43.0 34.0 '3' '0' '6' '5' '7' '17' '13' '15'
 '16' '10' '9' '8' '11' '14' '21' '22' '20' '18' '26' '25' '23' nan '28'
 '24' '31' '38' '29' '30' '33' '32' '34' '36' '37' '40' '39' '51' '41'
 '45' '44' '43' 4.0 nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan 36.0 nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan 41.0 32.0 40.0 nan nan nan nan 37.0 nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan 39.0 nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan 

In [28]:
# proportion of NA value of WindSpeed
ws_na_proportion = df['WindSpeed'].isna().mean()
print("Proportion of WindSpeed NA values:", ws_na_proportion)

Proportion of WindSpeed NA values: 0.00023539255061854575


In [29]:
# remove rows with WindSpeed is NA
df = df.dropna(subset=['WindSpeed'])

# convert WindSpeed to float type
df['WindSpeed'] = df['WindSpeed'].astype(float)

In [30]:
df.to_csv('csv_flight/preprocessed_df.csv', index=False)  

In [31]:
df.head()

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Reporting_Airline,Tail_Number,Flight_Number_Reporting_Airline,OriginAirportSeqID,...,CRSElapsedTime,Distance,DaysSince,is_holiday,is_day_before_holiday,is_holiday_week,MinSince,OriginFlightDensity,Visibility,WindSpeed
0,2014,1,1,1,3,2014-01-01,OO,N724SK,4974,1039705,...,141.0,689.0,0,1,0,1,360,2.0,10.0,5.0
1,2014,1,1,1,3,2014-01-01,AA,N002AA,1315,1039705,...,150.0,731.0,0,1,0,1,420,3.0,10.0,5.0
2,2014,1,1,1,3,2014-01-01,UA,N76288,1134,1039705,...,128.0,606.0,0,1,0,1,427,3.0,10.0,5.0
3,2014,1,1,1,3,2014-01-01,DL,N709DN,110,1039705,...,294.0,1947.0,0,1,0,1,490,6.0,10.0,3.0
4,2014,1,1,1,3,2014-01-01,DL,N654DL,1347,1039705,...,323.0,2139.0,0,1,0,1,495,6.0,10.0,3.0


In [32]:
# df = df.drop(['DaysSince', 'is_holiday', 'is_day_before_holiday', 'MinSince'], axis=1)